In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

import gtfs_utils

sns.set_style("white")
sns.set_context("talk")
sns.set_palette('Set2', 10)

In [3]:
FOLDER = 'data\\siri\\2018-08'

In [4]:
import os.path
s = pd.read_parquet(os.path.join(FOLDER, 'siri_rt_data.2018-08-01.0.log.parq'))
s.head()

,agency,bus_id,end_time,line_num,route_id,service_id,start_time,time_recorded,lat,lon,date
0,5,7591,660,19,2340,33441985,84300,86348,32.035385,34.767807,2018-08-01
1,5,7666,2340,19,2340,33643429,0,86109,0.000000,0.000000,2018-08-01
2,5,10196,300,16,2323,33425076,84000,86372,32.045105,34.809189,2018-08-01
3,5,7853,1200,16,2323,33425077,85200,86380,32.055714,34.781860,2018-08-01
4,5,10159,2100,16,2323,33425078,0,86109,0.000000,0.000000,2018-08-01


In [5]:
from dask.distributed import Client

In [6]:
#>>> client = Client('127.0.0.1:8786')
client = Client()

In [7]:
import dask.dataframe as dd

df = dd.read_parquet(FOLDER+'\\*.parq', index='service_id', engine='pyarrow')

In [22]:
import glob
rs = (pd.concat([pd.read_pickle(f, compression='gzip') 
                 for f in glob.glob('data\\gtfs_stats_MOD_ffill\\2018-08-*_route_stats.pkl.gz')])
      .assign(date = lambda x: pd.to_datetime(x.date))
      .assign(dayofweek = lambda x: x['date'].dt.strftime('%A')))

In [16]:
rs[(rs.route_short_name=='111') & (rs.route_long_name.str.contains('מודיעין'))].route_id.value_counts().index

Index(['9024', '11179', '11178', '21722', '20750'], dtype='object')

In [21]:
rs[(rs.route_id=='16561')]

AttributeError: 'Series' object has no attribute 'route_id'

In [8]:
#df.memory_usage().sum().compute()

In [9]:
#len(df) # 153997865

In [17]:
r = df[(df.lat!=0) & (df.route_id.isin([9024, 11179, 11178, 21722, 20750]))]# & (df.date>'2018-08-29')]#.drop_duplicates(subset=['bus_id', 'route_id', 'date', 'time_recorded'])
#r = client.persist(r)
r = r.repartition(npartitions=105)

In [18]:
rs = r.groupby('route_id').size().compute()

In [19]:
rs.head()

route_id
9024      9911
11178    59347
11179    47483
20750     4632
21722     2710
dtype: int64

In [15]:
rs.sort_values(ascending=False)

route_id
13428    359789
13429    357670
2533     277098
11685    247464
11689    245487
2532     243754
9811     191053
9813     185735
2567     175139
9803     169403
2457     168274
2566     164167
10797    161534
19740    160983
9802     160285
2454     155175
10804    155166
2328     146721
13435    143505
9816     140539
9808     139498
2332     137884
9817     137816
9821     135744
9822     135706
10799    134319
2510     134119
2492     133727
9809     132483
2961     131014
          ...  
4685         41
17442        41
16909        41
2890         40
11415        39
17993        38
23448        37
7837         36
8846         34
10465        34
11401        33
425          32
8110         32
23449        32
16908        31
426          27
19377        26
12653        24
11412        23
19153        23
23447        20
11414        17
4312         16
16058        15
11410        12
16009        11
11411        11
17170         9
11110         7
13599         5
Length: 6194, d

In [16]:
rs.sum() # 67 million without dedup, only remove zero coords

67832523

In [12]:
def chunk(s):
    '''
    The function applied to the
    individual partition (map)
    '''    
    return s.apply(lambda x: list(set(x)))


def agg(s):
    '''
    The function whic will aggrgate 
    the result from all the partitions(reduce)
    '''
    s = s._selected_obj    
    return s.groupby(level=list(range(s.index.nlevels))).sum()


def finalize(s):
    '''
    The optional functional that will be 
    applied to the result of the agg_tu functions
    '''
    return s.apply(lambda x: len(set(x)))


tunique = dd.Aggregation('tunique', chunk, agg,finalize)


In [13]:
d = r.groupby(['date', 'route_id']).agg({'time_recorded' : tunique}).compute()

distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Worker process 7964 was killed by signal 15
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Worker process 5280 was killed by signal 15
distributed.nanny - WARNING - Restarting worker
tornado.application - ERROR - Uncaught exception GET /status/ws?bokeh-protocol-version=1.0&bokeh-session-id=ricUmtV9gbFx3AG4MXglJPa1lhDrrry1DD91Ufp1sDkc (127.0.0.1)
HTTPServerRequest(protocol='http', host='localhost:8787', method='GET', uri='/status/ws?bokeh-protocol-version=1.0&bokeh-session-id=ricUmtV9gbFx3AG4MXglJPa1lhDrrry1DD91Ufp1sDkc', version='HTTP/1.1', remote_ip='127.0.0.1', headers={'Host': 'localhost:8787', 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:62.0) Gecko/20100101 Firefox/62.0', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8', 'Accept-L

tornado.application - ERROR - Future <tornado.concurrent.Future object at 0x0000029849CF18D0> exception was never retrieved: Traceback (most recent call last):
  File "C:\Users\dbare\Anaconda3\lib\site-packages\tornado\gen.py", line 1063, in run
    yielded = self.gen.throw(*exc_info)
  File "C:\Users\dbare\Anaconda3\lib\site-packages\bokeh\server\views\ws.py", line 211, in send_message
    yield message.send(self)
  File "C:\Users\dbare\Anaconda3\lib\site-packages\tornado\gen.py", line 1055, in run
    value = future.result()
  File "C:\Users\dbare\Anaconda3\lib\site-packages\tornado\concurrent.py", line 238, in result
    raise_exc_info(self._exc_info)
  File "<string>", line 4, in raise_exc_info
  File "C:\Users\dbare\Anaconda3\lib\site-packages\tornado\gen.py", line 1063, in run
    yielded = self.gen.throw(*exc_info)
  File "C:\Users\dbare\Anaconda3\lib\site-packages\bokeh\protocol\message.py", line 229, in send
    yield conn.write_message(self.header_json, locked=False)
  File "

tornado.application - ERROR - Future <tornado.concurrent.Future object at 0x00000298483AC278> exception was never retrieved: Traceback (most recent call last):
  File "C:\Users\dbare\Anaconda3\lib\site-packages\tornado\gen.py", line 1063, in run
    yielded = self.gen.throw(*exc_info)
  File "C:\Users\dbare\Anaconda3\lib\site-packages\bokeh\server\views\ws.py", line 211, in send_message
    yield message.send(self)
  File "C:\Users\dbare\Anaconda3\lib\site-packages\tornado\gen.py", line 1055, in run
    value = future.result()
  File "C:\Users\dbare\Anaconda3\lib\site-packages\tornado\concurrent.py", line 238, in result
    raise_exc_info(self._exc_info)
  File "<string>", line 4, in raise_exc_info
  File "C:\Users\dbare\Anaconda3\lib\site-packages\tornado\gen.py", line 1063, in run
    yielded = self.gen.throw(*exc_info)
  File "C:\Users\dbare\Anaconda3\lib\site-packages\bokeh\protocol\message.py", line 229, in send
    yield conn.write_message(self.header_json, locked=False)
  File "

tornado.application - ERROR - Future <tornado.concurrent.Future object at 0x0000029849DBBC18> exception was never retrieved: Traceback (most recent call last):
  File "C:\Users\dbare\Anaconda3\lib\site-packages\tornado\gen.py", line 1063, in run
    yielded = self.gen.throw(*exc_info)
  File "C:\Users\dbare\Anaconda3\lib\site-packages\bokeh\server\views\ws.py", line 211, in send_message
    yield message.send(self)
  File "C:\Users\dbare\Anaconda3\lib\site-packages\tornado\gen.py", line 1055, in run
    value = future.result()
  File "C:\Users\dbare\Anaconda3\lib\site-packages\tornado\concurrent.py", line 238, in result
    raise_exc_info(self._exc_info)
  File "<string>", line 4, in raise_exc_info
  File "C:\Users\dbare\Anaconda3\lib\site-packages\tornado\gen.py", line 1063, in run
    yielded = self.gen.throw(*exc_info)
  File "C:\Users\dbare\Anaconda3\lib\site-packages\bokeh\protocol\message.py", line 229, in send
    yield conn.write_message(self.header_json, locked=False)
  File "

tornado.application - ERROR - Future <tornado.concurrent.Future object at 0x0000029849EB6080> exception was never retrieved: Traceback (most recent call last):
  File "C:\Users\dbare\Anaconda3\lib\site-packages\tornado\gen.py", line 1063, in run
    yielded = self.gen.throw(*exc_info)
  File "C:\Users\dbare\Anaconda3\lib\site-packages\bokeh\server\views\ws.py", line 211, in send_message
    yield message.send(self)
  File "C:\Users\dbare\Anaconda3\lib\site-packages\tornado\gen.py", line 1055, in run
    value = future.result()
  File "C:\Users\dbare\Anaconda3\lib\site-packages\tornado\concurrent.py", line 238, in result
    raise_exc_info(self._exc_info)
  File "<string>", line 4, in raise_exc_info
  File "C:\Users\dbare\Anaconda3\lib\site-packages\tornado\gen.py", line 1063, in run
    yielded = self.gen.throw(*exc_info)
  File "C:\Users\dbare\Anaconda3\lib\site-packages\bokeh\protocol\message.py", line 229, in send
    yield conn.write_message(self.header_json, locked=False)
  File "

...

tornado.application - ERROR - Future <tornado.concurrent.Future object at 0x0000029849D73FD0> exception was never retrieved: Traceback (most recent call last):
  File "C:\Users\dbare\Anaconda3\lib\site-packages\tornado\gen.py", line 1063, in run
    yielded = self.gen.throw(*exc_info)
  File "C:\Users\dbare\Anaconda3\lib\site-packages\bokeh\server\views\ws.py", line 211, in send_message
    yield message.send(self)
  File "C:\Users\dbare\Anaconda3\lib\site-packages\tornado\gen.py", line 1055, in run
    value = future.result()
  File "C:\Users\dbare\Anaconda3\lib\site-packages\tornado\concurrent.py", line 238, in result
    raise_exc_info(self._exc_info)
  File "<string>", line 4, in raise_exc_info
  File "C:\Users\dbare\Anaconda3\lib\site-packages\tornado\gen.py", line 1063, in run
    yielded = self.gen.throw(*exc_info)
  File "C:\Users\dbare\Anaconda3\lib\site-packages\bokeh\protocol\message.py", line 229, in send
    yield conn.write_message(self.header_json, locked=False)
  File "

tornado.application - ERROR - Future <tornado.concurrent.Future object at 0x000002984837DCF8> exception was never retrieved: Traceback (most recent call last):
  File "C:\Users\dbare\Anaconda3\lib\site-packages\tornado\gen.py", line 1063, in run
    yielded = self.gen.throw(*exc_info)
  File "C:\Users\dbare\Anaconda3\lib\site-packages\bokeh\server\views\ws.py", line 211, in send_message
    yield message.send(self)
  File "C:\Users\dbare\Anaconda3\lib\site-packages\tornado\gen.py", line 1055, in run
    value = future.result()
  File "C:\Users\dbare\Anaconda3\lib\site-packages\tornado\concurrent.py", line 238, in result
    raise_exc_info(self._exc_info)
  File "<string>", line 4, in raise_exc_info
  File "C:\Users\dbare\Anaconda3\lib\site-packages\tornado\gen.py", line 1063, in run
    yielded = self.gen.throw(*exc_info)
  File "C:\Users\dbare\Anaconda3\lib\site-packages\bokeh\protocol\message.py", line 229, in send
    yield conn.write_message(self.header_json, locked=False)
  File "

tornado.application - ERROR - Future <tornado.concurrent.Future object at 0x0000029849EB2828> exception was never retrieved: Traceback (most recent call last):
  File "C:\Users\dbare\Anaconda3\lib\site-packages\tornado\gen.py", line 1063, in run
    yielded = self.gen.throw(*exc_info)
  File "C:\Users\dbare\Anaconda3\lib\site-packages\bokeh\server\views\ws.py", line 211, in send_message
    yield message.send(self)
  File "C:\Users\dbare\Anaconda3\lib\site-packages\tornado\gen.py", line 1055, in run
    value = future.result()
  File "C:\Users\dbare\Anaconda3\lib\site-packages\tornado\concurrent.py", line 238, in result
    raise_exc_info(self._exc_info)
  File "<string>", line 4, in raise_exc_info
  File "C:\Users\dbare\Anaconda3\lib\site-packages\tornado\gen.py", line 1063, in run
    yielded = self.gen.throw(*exc_info)
  File "C:\Users\dbare\Anaconda3\lib\site-packages\bokeh\protocol\message.py", line 229, in send
    yield conn.write_message(self.header_json, locked=False)
  File "

tornado.application - ERROR - Future <tornado.concurrent.Future object at 0x0000029849F44A90> exception was never retrieved: Traceback (most recent call last):
  File "C:\Users\dbare\Anaconda3\lib\site-packages\tornado\gen.py", line 1063, in run
    yielded = self.gen.throw(*exc_info)
  File "C:\Users\dbare\Anaconda3\lib\site-packages\bokeh\server\views\ws.py", line 211, in send_message
    yield message.send(self)
  File "C:\Users\dbare\Anaconda3\lib\site-packages\tornado\gen.py", line 1055, in run
    value = future.result()
  File "C:\Users\dbare\Anaconda3\lib\site-packages\tornado\concurrent.py", line 238, in result
    raise_exc_info(self._exc_info)
  File "<string>", line 4, in raise_exc_info
  File "C:\Users\dbare\Anaconda3\lib\site-packages\tornado\gen.py", line 1063, in run
    yielded = self.gen.throw(*exc_info)
  File "C:\Users\dbare\Anaconda3\lib\site-packages\bokeh\protocol\message.py", line 229, in send
    yield conn.write_message(self.header_json, locked=False)
  File "

KeyboardInterrupt: 

In [ ]:
c = r.groupby(['date', 'route_id']).size().compute()

In [ ]:
d.head()